Problem 1 part 1

In [34]:
import numpy as np
x1 = np.array([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
x2 = np.array([1.0,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0.0])
p = np.array([28.1,34.3,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])
m = np.matmul(p.transpose(),x1)/(np.matmul(x1.transpose(),x1))
print(m)



44.24415584415584


problem 1 part 2

In [117]:
import torch as t
from torch.autograd import Variable
import math as m

x = np.linspace(0,1,num=11)
x = t.from_numpy(x)
p_exp = np.array([28.1,34.3,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])
p_exp = t.from_numpy(p_exp)
A = Variable(t.tensor([1.0, 1.0]), requires_grad=True)
err = []
a = 0.01
for i in range(11):
    p[i] = x[i]*t.exp(A[0]*((A[1]*(1-x[i])/(A[0]*x[i]+A[1]*(1-x[i]))))**2)*17.4637\
        +(1-x[i])*t.exp(A[1]*((A[0]*x[i])/(A[0]*x[i]+A[1]*(1-x[i]))**2))*28.8241
    loss = ((x[i]*t.exp(A[0]*((A[1]*(1-x[i])/(A[0]*x[i]+A[1]*(1-x[i]))))**2)*17.4637\
        +(1-x[i])*t.exp(A[1]*((A[0]*x[i])/(A[0]*x[i]+A[1]*(1-x[i]))**2))*28.8241)\
            -p_exp[i])**2
    loss.backward()
    with t.no_grad():
        A -= a*A.grad
        A.grad.zero_()
        
print(A.data.numpy())
print(loss.data.numpy())


[1.344439  1.4184194]
0.0013176900000000482


problem 2

In [120]:
run gp.py
run plotters.py
